<h1>ASSEMBLING GENOMES WITH SPADES</h1>
<p>We employed Spades v. 3.13.0<sup>1</sup> and Scinet's Niagara computing resources<sup>2</sup>. The scripts which create the scinet jobs can be modified in order to make them usable in other computer systems. Alternatively, these experiments can be run locally, but these computations would take a long time.</p>
<p>The results are located at:
<ul>
<li>Assembled genomes can be found in <b>results/spades/ISOLATE_ID/ISOLATE_ID.fa</b> </li>
<li>Metrics of each assembly can be found in <b>results/spades/&lt;ISOLATE_ID&gt;/AssemblyMetrics.tsv</b></li>
<li>Summary of all assemblies can be found in <b>results/quast/AssemblySummary.tsv</b></li>
</ul>
</p>

In [ ]:
#Go into the spades directory
cd results/spades

<h2>RUNNING ASSEMBLER AT SCINET</h2>

In [ ]:
#Create jobs to be submitted to scinet's niagara
for i in `cat ../../doc/IdList.txt`; do echo -e '#!/bin/bash'"\n#SBATCH --nodes=1\n#SBATCH --cpus-per-task=40\n#SBATCH --time=1:00:00\n#SBATCH --job-name=spadesJob_"$i"\n#SBATCH --output=spadesJob_"$i".txt\n../../bin/SPAdes-3.13.0-Linux/bin/spades.py -t 40 --pe1-1 ../dedupper/"$i"_1.ddp.fq --pe1-2 ../dedupper/"$i"_2.ddp.fq -o "$i" 2> spades_"$i".err > spades_"$i".out" > spadesJob_$i.sh; done

In [ ]:
#Submit jobs for execution.
for i in `cat ../../doc/IdList.txt`; do sbatch "spadesJob_"$i".sh";done

<h2>OR RUNNING ASSEMBLER LOCALLY (THIS WILL TAKE A LONG TIME)</h2>

In [ ]:
for i in `cat ../../doc/IdList.txt`; do echo "../../bin/SPAdes-3.13.0-Linux/bin/spades.py --pe1-1 ../dedupper/"$i"_1.ddp.fq --pe1-2 ../dedupper/"$i"_2.ddp.fq -o "$i"; done

<h1>CHECKING RESULTS</h1>
<p>We employed quast v. 5.0.2<sup>3</sup> to evaluate the assembled genomes. Then, we summarize the results. These experiments are recorded to results/quast. </p>

<h2>RUNNING QUAST</h2>

In [ ]:
#Go into the quast directory
cd ../quast

In [ ]:
#Executing quast. I did not submit these as jobs to the SLURM because they are executed fastly.
for i in `cat ../../doc/IdList.txt`;do ../../bin/quast-5.0.2/quast.py -m 1000 ../spades/$i/scaffolds.fasta -o $i; done

<h2>SUMMARIZING RESULTS</h2>
<p>Here, I extract the assembly metrics calculated by quast and summarize them in a file called 'AssemblySummary.tsv'.</p>

In [ ]:
#Extract number of contigs, total length, and N50 for each isolate; then, save these stats in another file. 
echo -e "Isolate\tContigCount\tTotalLength\tN50" >> AssemblySummary.tsv
for i in `cat ../../doc/IdList.txt`;do contig=`sed '5q;d' $i/report.txt | awk '{print $6}'`; length=`sed '11q;d' $i/report.txt | awk '{print $6}'`; n50=`sed '20q;d' $i/report.txt | awk '{print $2}'`; echo -e $i"\t"$contig"\t"$length"\t"$n50;done >> AssemblySummary.tsv

<h1>FORMATTING ASSEMBLIES</h1>
<p>I created a summary of the assembly metrics of each de novo assembly, which was saved as AssemblyMetris.tsv. Contigs shoter than 1,000 bp were removed, and the fasta header of each contig was simplified.</p>

In [ ]:
#Go to the spades folder
cd ../spades

In [ ]:
#Remove contigs shorter than 1000 bp
for i in `cat ../../doc/IdList.txt`; do ../../bin/bioawk-master/bioawk -c fastx '{ if(length($seq) > 1000) { print ">"$name; print $seq }}' $i/scaffolds.fasta > $i/scaffolds_min1000.fasta; done

In [ ]:
#Create summary of contig metrics, which will be saved in the folder of each isolate as Assembly Metrics.tsv
for i in `cat ../../doc/IdList.txt`; do echo -e "ContigId\tContigLength\tContigCoverage" > $i/AssemblyMetrics.tsv; awk -F'_' -v I=$i '{if ($1 ~ /^>/) {print "'"$i"'_contig_"$2"\t"$4"\t"$6} }' $i/scaffolds_min1000.fasta >> $i/AssemblyMetrics.tsv; done

In [ ]:
#Rename contigs for simplicity
for i in `cat ../../doc/IdList.txt`;do awk -F'_' -v I=$i '{if ($1 ~ /^>/) {print ">'"$i"'_contig_"$2} else {print} }' $i/scaffolds_min1000.fasta > $i/$i.fa; done

<h4>NEXT STEPS</h4>
<p>Create figures to illustrate assemblies. This can be found at "<b>3a_CoveragePlots</b>"</p>

<sup>1</sup> http://cab.spbu.ru/software/spades/ 
<br><sup>2</sup> https://www.scinethpc.ca
<br><sup>3</sup> http://quast.bioinf.spbau.ru